In [4]:
import pandas as pd
from pathlib import Path

In [11]:
import pandas as pd
import hashlib
import unicodedata

# 🎯 Función para generar ID hash base36 de 6 caracteres
def generar_id_hash(texto):
    h = hashlib.md5(texto.encode('utf-8')).hexdigest()
    base10 = int(h, 16)
    base36 = ''
    chars = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    while base10 > 0:
        base10, i = divmod(base10, 36)
        base36 = chars[i] + base36
    return base36[:6].upper().zfill(6)

# 🔧 Normaliza (quita tildes, espacios y convierte a minúsculas)
def normalizar(texto):
    texto = str(texto).lower().strip()
    texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')
    return texto

# 🗂 Ruta del CSV
ruta_csv = r'H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS\data\raw\departamento salubridad NY\rows2.csv'
df = pd.read_csv(ruta_csv)

# 🔍 Extraer nombres únicos
nombres_originales = df['nombre'].dropna().drop_duplicates().sort_values().reset_index(drop=True)

# 🧼 Guardar los nombres normalizados (minúsculas + sin acento)
nombres_minusculas = nombres_originales.apply(lambda x: normalizar(x))

# 🔐 Generar los IDs hash desde los nombres normalizados
ids_hash = nombres_minusculas.apply(generar_id_hash)

# 🧱 Construir la tabla dimensión con los nombres ya en minúsculas
dim_nombres = pd.DataFrame({
    'id_nombre': ids_hash,
    'nombre': nombres_minusculas  # AQUÍ se guardan ya en minúsculas
})

# 💾 Guardar
ruta_salida = r'H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS\data\processed\dim\dim_nombre_restaurante.csv'
dim_nombres.to_csv(ruta_salida, index=False, encoding='utf-8-sig')

print(f"✅ Nombres guardados en minúsculas y IDs generados correctamente.\n📁 Archivo listo en:\n{ruta_salida}")



✅ Nombres guardados en minúsculas y IDs generados correctamente.
📁 Archivo listo en:
H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS\data\processed\dim\dim_nombre_restaurante.csv


C:\Users\jrgsi\AppData\Local\Temp\ipykernel_13432\1805473131.py:24: DtypeWarning: Columns (3,4,5,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta_csv)


In [1]:
# -*- coding: utf-8 -*-
# Script DEDICADO a la limpieza, normalización y análisis de duplicados
# de nuestra tabla de dimensión de nombres de restaurantes. ¡Vamos a dejarla impecable!

# --- Importaciones Esenciales ---
# Cargamos nuestras herramientas principales
import pandas as pd           # Nuestro caballo de batalla para manejar datos
import numpy as np            # Para operaciones numéricas y el útil NaN (Not a Number)
import re                     # Expresiones regulares, ¡geniales para buscar patrones en texto!
import unicodedata            # Para manejar caracteres especiales y acentos
import logging                # Nuestro diario de a bordo para saber qué hace el script
from pathlib import Path      # Para manejar rutas de archivo de forma moderna y compatible
import sys                    # Para poder detener el script si algo va muy mal
from io import StringIO       # Para capturar info() en logs (opcional pero útil)

# --- Configuración de Logging (Nuestro Diario de a Bordo) ---
# Configuramos cómo queremos ver los mensajes mientras corre el script.
logging.basicConfig(
    level=logging.INFO, # Mostraremos mensajes informativos, de advertencia y errores.
    format='%(asctime)s [%(levelname)s] %(name)s.%(funcName)s: %(message)s', # Formato detallado
    datefmt='%Y-%m-%d %H:%M:%S' # Formato de fecha y hora local
)
log = logging.getLogger(__name__) # Creamos un logger específico para este script.

# --- Definición de Rutas (El Mapa del Tesoro) ---
# Centralizamos todas las rutas aquí para encontrarlas fácil.
try:
    # 1. RUTA_BASE: La carpeta principal de nuestro proyecto.
    #    *** ¡OJO! Revisa que esta ruta sea correcta en TU computadora. ***
    RUTA_BASE = Path(r"H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS").resolve()

    # 2. Subcarpetas estándar (las creamos si no existen más abajo).
    RUTA_PROCESSED = RUTA_BASE / "data" / "processed" # Donde van nuestros datos limpios.
    RUTA_DIM = RUTA_PROCESSED / "dim"                # Donde van nuestras tablas de dimensión.

    # 3. Ruta a la Dimensión de Nombres (¡Nuestro objetivo!)
    #    Este es el archivo que vamos a leer y LUEGO sobreescribir con la versión limpia.
    #    *** ¡Asegúrate de que el nombre del archivo sea el correcto! ***
    RUTA_DIM_NOMBRES = (RUTA_DIM / "dim_nombre_restaurante_limpia.csv").resolve()
    # Si se llama diferente, ajústalo aquí:
    # RUTA_DIM_NOMBRES = (RUTA_DIM / "dim_nombre_restaurante.csv").resolve()

    # 4. Ruta para el Reporte de Duplicados que generaremos.
    RUTA_REPORTE_DUPLICADOS = (RUTA_PROCESSED / "posibles_duplicados_nombres.csv").resolve()

    # 5. Crear las carpetas si no existen (¡así no falla el guardado!)
    RUTA_PROCESSED.mkdir(parents=True, exist_ok=True)
    RUTA_DIM.mkdir(parents=True, exist_ok=True)

except Exception as e:
    log.exception(f"¡Error crítico! No pudimos definir o resolver las rutas. Revisa la RUTA_BASE: {e}")
    sys.exit(1) # Detenemos el script si no podemos ni definir las rutas.

# --- Funciones de Limpieza y Normalización ---

def normalizar_nombre_restaurante(nombre: str) -> str | None:
    """
    Nuestra función "pulidora" de nombres. Intenta estandarizar al máximo
    para encontrar duplicados que se escriben diferente.

    Args:
        nombre (str): El nombre original del restaurante.

    Returns:
        str or None: El nombre normalizado, o None si no es válido.
    """
    # Si no es texto o es nulo, no podemos hacer nada.
    if pd.isna(nombre) or not isinstance(nombre, str):
        return None

    try:
        # 1. A minúsculas y quitamos espacios al inicio/final.
        nombre_norm = nombre.lower().strip()

        # 2. Quitar comillas comunes al inicio/final (simples, dobles, triples).
        nombre_norm = re.sub(r'^["\']{1,3}|["\']{1,3}$', '', nombre_norm).strip()

        # 3. Quitar caracteres especiales comunes al inicio (ej. $1 Pizza -> 1 Pizza).
        #    Hay que tener cuidado para no quitar algo importante.
        nombre_norm = re.sub(r'^[#$*%\+\?¿!¡\-_]+', '', nombre_norm).strip()

        # 4. Normalización Unicode: Adiós acentos y caracteres "raros".
        nombre_norm = unicodedata.normalize('NFKD', nombre_norm).encode('ASCII', 'ignore').decode('utf-8')

        # 5. Eliminar sufijos comunes de negocios y tipos de lugar.
        #    Usamos \b para asegurar que sean palabras completas (ej. no quitar "inc" de "zinc").
        sufijos_comunes = [
            r'\bllc\b', r'\bltd\b', r'\binc\b', r'\bcorp\b', r'\bcorporation\b',
            r'\brestaurant\b', r'\bcafe\b', r'\bpizzeria\b', r'\bgrill\b',
            r'\bbar\b', r'\bdeli\b', r'\bexpress\b', r'\bkitchen\b',
            r'\bthe\b' # "the" como palabra suelta.
        ]
        for sufijo in sufijos_comunes:
            # flags=re.IGNORECASE para que no importe si es LLC o llc.
            nombre_norm = re.sub(sufijo, '', nombre_norm, flags=re.IGNORECASE)

        # 6. Manejo de Puntuación: Quitamos casi todo, pero intentamos mantener apóstrofes internos (como en Joe's).
        nombre_norm = nombre_norm.replace("'", "__APOSTROPHE__") # Reemplazo temporal.
        # Quitamos todo lo que NO sea letra, número, espacio o nuestro placeholder.
        nombre_norm = re.sub(r'[^\w\s__APOSTROPHE__]+', '', nombre_norm.replace('_',' '))
        nombre_norm = nombre_norm.replace("__APOSTROPHE__", "'") # Lo restauramos.

        # 7. Normalizar espacios: Múltiples espacios se vuelven uno, y quitamos inicio/final.
        nombre_norm = re.sub(r'\s+', ' ', nombre_norm).strip()

        # 8. Filtrar nombres que claramente no son válidos o son ruido.
        placeholders_ruido = {'name', '#name?', '', 'test'}
        # Si el nombre normalizado está en la lista, o si consiste SOLO en símbolos como * o #...
        if nombre_norm in placeholders_ruido or re.match(r'^[\*#]+$', nombre_norm):
            return None # ... lo consideramos inválido.

        # Si sobrevivió a todo y no está vacío, ¡lo devolvemos!
        return nombre_norm if nombre_norm else None

    except Exception as e:
        # Si algo falla en el proceso, avisamos y devolvemos el original.
        log.warning(f"Error normalizando el nombre '{nombre}': {e}")
        return nombre

def identificar_y_reportar_duplicados(df_dim: pd.DataFrame, ruta_reporte: Path) -> pd.DataFrame:
    """
    Esta función es nuestro detective de duplicados.
    1. Añade la columna 'nombre_normalizado' al DataFrame.
    2. Busca grupos de nombres normalizados que tengan MÁS DE UN 'id_nombre' diferente.
    3. Guarda un reporte en CSV con estos grupos sospechosos.
    4. Devuelve el DataFrame con la columna 'nombre_normalizado' (la quitaremos después).

    Args:
        df_dim (pd.DataFrame): DataFrame de la dimensión de nombres (ya con limpieza básica).
        ruta_reporte (Path): Dónde guardar el reporte de duplicados.

    Returns:
        pd.DataFrame: El mismo DataFrame pero con la columna 'nombre_normalizado' añadida.
    """
    log.info("Iniciando normalización y búsqueda de posibles duplicados...")
    col_nombre = 'nombre'; col_id = 'id_nombre'; col_norm = 'nombre_normalizado'
    col_flag = 'permiso_temporal' # Para incluir en el reporte

    # Verificación rápida: ¿Tenemos las columnas necesarias?
    if not all(c in df_dim.columns for c in [col_nombre, col_id]):
        log.error(f"¡Faltan columnas '{col_nombre}' o '{col_id}'! No podemos buscar duplicados.")
        # Añadimos la columna normalizada como vacía para no romper el flujo
        df_dim[col_norm] = None
        return df_dim

    # Aplicamos la normalización a toda la columna 'nombre'
    log.info(f"Aplicando función de normalización a '{col_nombre}'...")
    df_dim[col_norm] = df_dim[col_nombre].apply(normalizar_nombre_restaurante)
    nulos_norm = df_dim[col_norm].isnull().sum()
    if nulos_norm > 0:
        log.warning(f"{nulos_norm} nombres resultaron nulos/vacíos tras normalizar (serán ignorados en la detección).")

    # ¡La búsqueda! Agrupamos por nombre normalizado y filtramos los grupos
    # donde haya más de un id_nombre único. Usamos dropna para ignorar los nulos.
    duplicados_agrupados = df_dim.dropna(subset=[col_norm, col_id])\
                               .groupby(col_norm)\
                               .filter(lambda grupo: grupo[col_id].nunique() > 1)

    if duplicados_agrupados.empty:
        log.info("¡Buenas noticias! No se encontraron grupos con IDs potencialmente duplicados por nombre normalizado.")
        # Podríamos borrar un reporte viejo aquí si quisiéramos.
    else:
        num_grupos = duplicados_agrupados[col_norm].nunique()
        log.warning(f"¡Ojo! Encontramos {num_grupos} grupos de nombres que parecen ser el mismo restaurante pero tienen IDs diferentes.")
        # Preparamos el reporte para que sea fácil de revisar
        columnas_reporte = [col_norm, col_nombre, col_id]
        if col_flag in duplicados_agrupados.columns: # Incluir el flag si existe
            columnas_reporte.append(col_flag)
        reporte = duplicados_agrupados[columnas_reporte].sort_values(by=[col_norm, col_nombre])
        try:
            # Guardamos el reporte
            reporte.to_csv(ruta_reporte, index=False, encoding='utf-8-sig')
            log.info(f"Reporte de duplicados potenciales guardado en: {ruta_reporte}")
        except Exception as e:
            log.exception(f"¡Error! No se pudo guardar el reporte de duplicados: {e}")

    log.info("Normalización e identificación de duplicados completada.")
    # Devolvemos el DataFrame original con la columna normalizada añadida temporalmente
    return df_dim

def guardar_dimension(dim_df: pd.DataFrame, ruta_dim: Path):
    """
    Guarda nuestro DataFrame de dimensión final en formato CSV.
    Se asegura de que la columna de flag exista y tenga el tipo correcto.
    Asume que las columnas ya están seleccionadas y ordenadas como queremos.

    Args:
        dim_df (pd.DataFrame): El DataFrame de dimensión listo para guardar.
        ruta_dim (Path): La ruta completa del archivo CSV de salida.
    """
    log.info(f"Guardando dimensión final ({len(dim_df)} filas) en: {ruta_dim}")
    if dim_df is None or dim_df.empty:
        log.warning("El DataFrame de dimensión está vacío, no se guardará.")
        return

    # Doble chequeo y corrección del tipo de 'permiso_temporal'
    col_flag = 'permiso_temporal'
    if col_flag not in dim_df.columns:
        log.warning(f"Añadiendo columna '{col_flag}' faltante con default=False antes de guardar.")
        dim_df[col_flag] = False
    try:
        # Forzar a booleano nullable de Pandas para consistencia
        dim_df[col_flag] = dim_df[col_flag].astype('boolean')
    except Exception as e:
        log.warning(f"No se pudo convertir '{col_flag}' a booleano, guardando como objeto: {e}")
        # Como fallback, aseguramos que no haya NaNs problemáticos para CSV
        dim_df[col_flag] = dim_df[col_flag].fillna(False)

    try:
        # Guardamos el archivo CSV. index=False es importante. utf-8-sig ayuda con Excel.
        dim_df.to_csv(ruta_dim, index=False, encoding='utf-8-sig')
        log.info(f"¡Dimensión guardada exitosamente!")
    except Exception as e:
        # ¡Si esto falla, es un problema!
        log.exception(f"¡ERROR CRÍTICO! No se pudo guardar la dimensión en '{ruta_dim}': {e}")


# --- Función Principal de Limpieza (Nuestro Orquestador) ---
def main_limpieza_dim_nombres():
    """
    Orquesta todo el proceso: Carga la dimensión, aplica limpieza básica,
    normaliza nombres, reporta duplicados, ordena y guarda la versión final.
    """
    log.info("================================================")
    log.info("=== INICIO Limpieza Dimensión Nombres Restaurante ===")
    log.info("================================================")

    # 1. Cargar Dimensión Existente
    # -------------------------------
    log.info(f"Paso 1: Cargando dimensión desde: {RUTA_DIM_NOMBRES}")
    if not RUTA_DIM_NOMBRES.is_file():
        log.critical(f"¡Abortando! No se encontró el archivo de dimensión: {RUTA_DIM_NOMBRES}")
        sys.exit(1) # Salimos si no existe el archivo que queremos limpiar
    try:
        # Le decimos a Pandas que trate IDs y nombres como texto al cargar, y el flag como objeto por ahora
        dtype_map = {'id_nombre': str, 'nombre': str, 'permiso_temporal': object}
        # keep_default_na=False ayuda a leer valores booleanos que podrían estar como texto 'True'/'False'
        dim_nombres = pd.read_csv(RUTA_DIM_NOMBRES, dtype=dtype_map, keep_default_na=False)
        log.info(f"Dimensión cargada: {dim_nombres.shape[0]} filas, {dim_nombres.shape[1]} columnas.")
    except Exception as e:
        log.exception(f"¡Error crítico al cargar la dimensión! {e}")
        sys.exit(1)
    if dim_nombres.empty:
        log.warning("La dimensión está vacía. No hay nada que limpiar. Saliendo.")
        return # Terminamos si no hay datos

    # 2. Limpieza Básica Previa (¡Más Robusta!)
    # -----------------------------------------
    log.info("Paso 2: Realizando limpieza básica inicial...")
    original_rows = len(dim_nombres) # Guardamos el número inicial de filas

    # 2a. Asegurar columna y tipo 'permiso_temporal'
    col_flag = 'permiso_temporal'
    if col_flag not in dim_nombres.columns:
        dim_nombres[col_flag] = False # La creamos si no existe
        log.warning(f"Columna '{col_flag}' no existía, se añadió con valor False.")
    # Convertimos a booleano de forma segura, manejando varios formatos de texto/número
    map_bool = {'true': True, 'false': False, '1': True, '0': False, '1.0': True, '0.0': False,
                True: True, False: False, np.nan: False, None: False, '': False, 'nan': False}
    dim_nombres[col_flag] = dim_nombres[col_flag].astype(str).str.lower().map(map_bool).fillna(False).astype('boolean')
    log.info(f"Columna '{col_flag}' asegurada como tipo Boolean (soporta nulos si los hubiera).")

    # 2b. Eliminar filas con IDs o Nombres inválidos/vacíos/ruidosos ANTES de quitar duplicados
    log.info("Filtrando filas con IDs o nombres inválidos/vacíos/ruidosos...")
    # Quitar nulos explícitos
    dim_nombres.dropna(subset=['id_nombre', 'nombre'], inplace=True)
    # Quitar los que quedaron como strings vacíos después de cargar/convertir
    dim_nombres = dim_nombres[dim_nombres['nombre'].astype(str).str.strip() != '']
    dim_nombres = dim_nombres[dim_nombres['id_nombre'].astype(str).str.strip() != '']
    # Filtrar IDs que no sigan el formato NOM + 6 Hexadecimales
    dim_nombres = dim_nombres[dim_nombres['id_nombre'].astype(str).str.match(r'^NOM[A-F0-9]{6}$')]
    # Filtrar nombres que parezcan solo ruido (símbolos/placeholders)
    placeholders_ruido = ['#NAME?', 'NAME', 'TEST']
    dim_nombres = dim_nombres[~dim_nombres['nombre'].astype(str).str.match(r'^[#\*$%\?¿!¡\-_<>",\s]+$')] # Solo símbolos/espacios
    dim_nombres = dim_nombres[~dim_nombres['nombre'].astype(str).str.upper().isin(placeholders_ruido)]
    rows_dropped_invalid = original_rows - len(dim_nombres)
    if rows_dropped_invalid > 0:
        log.info(f"Eliminadas {rows_dropped_invalid} filas con IDs/nombres inválidos/vacíos/ruidosos.")
    original_rows = len(dim_nombres) # Actualizamos el contador para el siguiente paso

    # 2c. Eliminar duplicados por nombre (ignorando mayúsculas/minúsculas y espacios extra)
    log.info("Eliminando duplicados por 'nombre' (insensible a mayús/minús, sin espacios extra)...")
    # Creamos una columna temporal con el nombre limpio para comparar
    dim_nombres['nombre_clean_temp'] = dim_nombres['nombre'].astype(str).str.strip().str.lower()
    # Eliminamos duplicados basados en esta columna limpia, manteniendo la ÚLTIMA aparición
    # (consistente con cómo los ETLs podrían haber actualizado el flag permiso_temporal)
    dim_nombres.drop_duplicates(subset=['nombre_clean_temp'], keep='last', inplace=True, ignore_index=True)
    # Eliminamos la columna temporal que ya no necesitamos
    dim_nombres.drop(columns=['nombre_clean_temp'], inplace=True)
    rows_dropped_duplicates = original_rows - len(dim_nombres)
    if rows_dropped_duplicates > 0:
        log.info(f"Eliminados {rows_dropped_duplicates} duplicados por 'nombre' (limpio, case-insensitive).")

    # Volvemos a verificar si nos quedamos sin datos después de limpiar
    if dim_nombres.empty:
        log.warning("La dimensión quedó vacía tras la limpieza básica. No se puede continuar.")
        return

    # 3. Normalizar Nombres y Reportar Duplicados Potenciales
    # -------------------------------------------------------
    # Esta función AÑADE la columna 'nombre_normalizado' temporalmente para el análisis.
    log.info("Paso 3: Normalizando nombres y generando reporte de duplicados...")
    dim_con_norm = identificar_y_reportar_duplicados(dim_nombres.copy(), RUTA_REPORTE_DUPLICADOS)
    # Usamos una copia por si la función modificara algo inesperadamente.

    # 4. Ordenar Alfabéticamente
    # ---------------------------
    log.info("Paso 4: Ordenando la dimensión alfabéticamente por 'nombre'...")
    col_orden = 'nombre'
    if col_orden in dim_con_norm.columns:
        dim_con_norm.sort_values(
            by=col_orden,
            inplace=True,
            ignore_index=True, # Resetea el índice final a 0, 1, 2...
            na_position='last', # Por si algún nombre quedó nulo (no debería pasar)
            key=lambda col: col.astype(str).str.lower() # Orden case-insensitive
        )
        log.info("Dimensión ordenada.")
    else:
        log.warning(f"No se pudo ordenar, falta la columna '{col_orden}'.")

    # 5. Preparar para Guardar (¡Quitando la columna normalizada!)
    # -----------------------------------------------------------
    log.info("Paso 5: Preparando DataFrame final para guardar...")
    col_a_quitar = 'nombre_normalizado'
    if col_a_quitar in dim_con_norm.columns:
        # Creamos el DataFrame final SIN la columna normalizada
        dim_a_guardar = dim_con_norm.drop(columns=[col_a_quitar])
        log.info(f"Columna '{col_a_quitar}' eliminada para el archivo final.")
    else:
        dim_a_guardar = dim_con_norm # Si no existía, usamos el DF tal cual
        log.warning(f"Columna '{col_a_quitar}' no encontrada para eliminar.")

    # Reordenamos las columnas que SÍ queremos guardar
    cols_dim_final = ['id_nombre', 'nombre', 'permiso_temporal']
    # Seleccionamos solo esas columnas en ese orden (si existen)
    dim_final_guardar = dim_a_guardar[[col for col in cols_dim_final if col in dim_a_guardar.columns]]
    log.info(f"Columnas finales seleccionadas para guardar: {dim_final_guardar.columns.tolist()}")

    # 6. Guardar la Dimensión Limpia y Ordenada
    # -----------------------------------------
    log.info("Paso 6: Guardando la dimensión final...")
    # Llamamos a nuestra función de guardado pasándole el DF ya preparado
    guardar_dimension(dim_final_guardar, RUTA_DIM_NOMBRES) # Sobreescribimos el archivo original

    # --- Fin del Proceso ---
    log.info("================================================")
    log.info("=== Limpieza Dimensión Nombres Completada ===")
    # Informamos si se generó el reporte de duplicados
    if RUTA_REPORTE_DUPLICADOS.is_file(): # Verificamos si existe
        log.info(f"=== Reporte de posibles duplicados generado en: {RUTA_REPORTE_DUPLICADOS} ===")
        log.info("=== ¡Revisa ese archivo para decidir cómo unificar los IDs! ===")
    else:
        log.info("=== No se generó reporte de duplicados (probablemente no se encontraron). ===")
    log.info("================================================")


# --- Punto de Entrada ---
if __name__ == "__main__":
    # Esto asegura que main_limpieza_dim_nombres() solo se ejecute
    # cuando corres este script directamente.
    main_limpieza_dim_nombres()

2025-04-19 12:34:00 [INFO] __main__.main_limpieza_dim_nombres: ================================================
2025-04-19 12:34:00 [INFO] __main__.main_limpieza_dim_nombres: === INICIO Limpieza Dimensión Nombres Restaurante ===
2025-04-19 12:34:00 [INFO] __main__.main_limpieza_dim_nombres: ================================================
2025-04-19 12:34:00 [INFO] __main__.main_limpieza_dim_nombres: Paso 1: Cargando dimensión desde: H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS\data\processed\dim\dim_nombre_restaurante_limpia.csv
2025-04-19 12:34:00 [INFO] __main__.main_limpieza_dim_nombres: Dimensión cargada: 49322 filas, 3 columnas.
2025-04-19 12:34:00 [INFO] __main__.main_limpieza_dim_nombres: Paso 2: Realizando limpieza básica inicial...
2025-04-19 12:34:01 [INFO] __main__.main_limpieza_dim_nombres: Columna 'permiso_temporal' asegurada como tipo Boolean (soporta nulos si los hubiera).
2025-04-19 12:34:01 [INFO] __main__.main_limpieza_dim_nombres: Filtrando filas con IDs o nombre